In [2]:
import time
import datetime
import pickle
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

In [4]:
#Se carga la lista que contiene los datos meteorológicos

fh = open("aemet_2015_2021.pkl", "rb")
lista_datos = pickle.load(fh)
fh.close()

In [5]:
#Se carga el diccionario estaciones_provincia

fh = open("estaciones_provincia.pkl", "rb")
estaciones_provincia = pickle.load(fh)
fh.close()

estaciones_provincia

{'4358X': 'BADAJOZ',
 '4220X': 'CIUDAD REAL',
 'C447A': 'STA. CRUZ DE TENERIFE',
 '6106X': 'MALAGA',
 '9698U': 'LLEIDA',
 '4410X': 'BADAJOZ',
 '1331A': 'ASTURIAS',
 '1690A': 'OURENSE',
 '0002I': 'TARRAGONA',
 '8489X': 'CASTELLON',
 '8025': 'ALICANTE',
 'C449C': 'STA. CRUZ DE TENERIFE',
 '9784P': 'HUESCA',
 '2331': 'BURGOS',
 '9434': 'ZARAGOZA',
 '1109': 'CANTABRIA',
 '6293X': 'ALMERIA',
 '1212E': 'ASTURIAS',
 '3094B': 'CUENCA',
 '1059X': 'BIZKAIA',
 '1207U': 'ASTURIAS',
 '6367B': 'ALMERIA',
 '7228': 'MURCIA',
 '3168C': 'GUADALAJARA',
 '7002Y': 'MURCIA',
 'C649I': 'LAS PALMAS',
 '3191E': 'MADRID',
 '0016A': 'TARRAGONA',
 '8058X': 'VALENCIA',
 '0367': 'GIRONA',
 '7247X': 'ALICANTE',
 '3200': 'MADRID',
 '4103X': 'CIUDAD REAL',
 '3175': 'MADRID',
 '3129': 'MADRID',
 '4067': 'TOLEDO',
 'C689E': 'LAS PALMAS',
 '1014A': 'GIPUZKOA',
 '7031X': 'MURCIA',
 '7209': 'MURCIA',
 '3519X': 'CACERES',
 '2755X': 'ZAMORA',
 '8175': 'ALBACETE',
 '9244X': 'ZARAGOZA',
 '6277B': 'ALMERIA',
 '9170': 'LA RIOJA'

# PREPROCESAMIENTO

### Se preprocesa el archivo que contiene los datos del precio de la electricidad por fecha

In [64]:
#Se carga el dataframe con la información del precio de la electricidad

df = pd.read_csv("PrecioMedioHorarioFinalSumaDeComponentes_2015-2021.csv", delimiter = ";")
df

,id,name,geoid,geoname,value,datetime
0,10211,Precio medio horario final suma de componentes,NaN,NaN,65.41,2015-01-01T00:00:00+01:00
1,10211,Precio medio horario final suma de componentes,NaN,NaN,64.92,2015-01-01T01:00:00+01:00
2,10211,Precio medio horario final suma de componentes,NaN,NaN,64.48,2015-01-01T02:00:00+01:00
3,10211,Precio medio horario final suma de componentes,NaN,NaN,59.32,2015-01-01T03:00:00+01:00
4,10211,Precio medio horario final suma de componentes,NaN,NaN,56.04,2015-01-01T04:00:00+01:00
...,...,...,...,...,...,...
54019,10211,Precio medio horario final suma de componentes,NaN,NaN,42.19,2021-02-28T19:00:00+01:00
54020,10211,Precio medio horario final suma de componentes,NaN,NaN,47.78,2021-02-28T20:00:00+01:00
54021,10211,Precio medio horario final suma de componentes,NaN,NaN,46.27,2021-02-28T21:00:00+01:00
54022,10211,Precio medio horario final suma de componentes,NaN,NaN,37.03,2021-02-28T22:00:00+01:00


In [65]:
#Se eliminan las columnas "id", "name", "geoid" y "geoname", ya que no aportan nada de información

df.drop(["id", "name", "geoid", "geoname"], axis = 1, inplace = True)
df

,value,datetime
0,65.41,2015-01-01T00:00:00+01:00
1,64.92,2015-01-01T01:00:00+01:00
2,64.48,2015-01-01T02:00:00+01:00
3,59.32,2015-01-01T03:00:00+01:00
4,56.04,2015-01-01T04:00:00+01:00
...,...,...
54019,42.19,2021-02-28T19:00:00+01:00
54020,47.78,2021-02-28T20:00:00+01:00
54021,46.27,2021-02-28T21:00:00+01:00
54022,37.03,2021-02-28T22:00:00+01:00


In [66]:
#Se separa la variable "datetime" en dos variables: "date" y "time"

df["date"] = df["datetime"].map(lambda x : x.split("T")[0])
df["time"] = df["datetime"].map(lambda x : x.split("T")[1].split(":")[0])
df.drop("datetime", axis = 1, inplace = True)
df

,value,date,time
0,65.41,2015-01-01,00
1,64.92,2015-01-01,01
2,64.48,2015-01-01,02
3,59.32,2015-01-01,03
4,56.04,2015-01-01,04
...,...,...,...
54019,42.19,2021-02-28,19
54020,47.78,2021-02-28,20
54021,46.27,2021-02-28,21
54022,37.03,2021-02-28,22


In [67]:
#Se crea una variable que indica el día de la semana (desde el 1, que es lunes, hasta el 7, que es domingo)

for i in range(len(df)):
    df.loc[i, "weekday"] = datetime.datetime.strptime(df.loc[i,"date"], "%Y-%m-%d").isoweekday()
    
df["weekday"] = df["weekday"].astype("int8")
df

,value,date,time,weekday
0,65.41,2015-01-01,00,4
1,64.92,2015-01-01,01,4
2,64.48,2015-01-01,02,4
3,59.32,2015-01-01,03,4
4,56.04,2015-01-01,04,4
...,...,...,...,...
54019,42.19,2021-02-28,19,7
54020,47.78,2021-02-28,20,7
54021,46.27,2021-02-28,21,7
54022,37.03,2021-02-28,22,7


### Se obtiene un único dataframe conjunto de electricidad y datos meteorológicos

In [68]:
#Se crea un set que contenga los id de todas las estaciones meteorológicas

estaciones = []
for i in lista_datos:
    for j in i:
        estaciones.append(j["indicativo"])
        
set_estaciones = set(estaciones)

In [69]:
#Se amplía el "esqueleto" del dataframe, añadiéndo las siguientes variables:
#tmed (temperautra media del día)
#prec (precipitaciones totales del día)
#velmedia (velocidad media del viento del día)
#sol (horas de sol del día)
#Cada estación meteorológica (representada por su indicativo) tiene estas cuatro variables (es decir, se han creado las cuatro variables por cada estación)

for i in set_estaciones:
    for j in ["tmed", "prec", "velmedia", "sol"]:
        variable = str(i + "-" + j)
        df[variable] = np.nan

df

,value,date,time,weekday,2867-tmed,2867-prec,2867-velmedia,2867-sol,1083L-tmed,1083L-prec,...,6325O-velmedia,6325O-sol,B569X-tmed,B569X-prec,B569X-velmedia,B569X-sol,9576C-tmed,9576C-prec,9576C-velmedia,9576C-sol
0,65.41,2015-01-01,00,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64.92,2015-01-01,01,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64.48,2015-01-01,02,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59.32,2015-01-01,03,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,56.04,2015-01-01,04,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54019,42.19,2021-02-28,19,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54020,47.78,2021-02-28,20,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54021,46.27,2021-02-28,21,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54022,37.03,2021-02-28,22,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
%%time

#Se rellenan las variables creadas anteriormente a partir de los datos de la lista meteorológica

for z in lista_datos:
    for i in z:
        id_estacion = i["indicativo"]
        fecha = i["fecha"]
        indices = df[df["date"] == fecha].index
        
        try:
            tmed = float(i["tmed"].replace(",", "."))
            for j in indices:
                df.loc[j,(id_estacion + "-" + "tmed")] = tmed
        except:
            pass

        try:
            prec = float(i["prec"].replace(",", "."))
            for j in indices:
                df.loc[j,(id_estacion + "-" + "prec")] = prec
        except:
            pass

        try:
            velmedia = float(i["velmedia"].replace(",", "."))
            for j in indices:
                df.loc[j,(id_estacion + "-" + "velmedia")] = velmedia
        except:
            pass

        try:
            sol = float(i["sol"].replace(",", "."))
            for j in indices:
                df.loc[j,(id_estacion + "-" + "sol")] = sol
        except:
            pass
    
df

Wall time: 5h 26min 9s


,value,date,time,weekday,2867-tmed,2867-prec,2867-velmedia,2867-sol,1083L-tmed,1083L-prec,...,6325O-velmedia,6325O-sol,B569X-tmed,B569X-prec,B569X-velmedia,B569X-sol,9576C-tmed,9576C-prec,9576C-velmedia,9576C-sol
0,65.41,2015-01-01,00,4,2.0,0.0,1.1,9.1,9.8,0.0,...,3.9,8.3,12.6,0.0,6.9,NaN,NaN,0.0,2.2,NaN
1,64.92,2015-01-01,01,4,2.0,0.0,1.1,9.1,9.8,0.0,...,3.9,8.3,12.6,0.0,6.9,NaN,NaN,0.0,2.2,NaN
2,64.48,2015-01-01,02,4,2.0,0.0,1.1,9.1,9.8,0.0,...,3.9,8.3,12.6,0.0,6.9,NaN,NaN,0.0,2.2,NaN
3,59.32,2015-01-01,03,4,2.0,0.0,1.1,9.1,9.8,0.0,...,3.9,8.3,12.6,0.0,6.9,NaN,NaN,0.0,2.2,NaN
4,56.04,2015-01-01,04,4,2.0,0.0,1.1,9.1,9.8,0.0,...,3.9,8.3,12.6,0.0,6.9,NaN,NaN,0.0,2.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54019,42.19,2021-02-28,19,7,6.0,0.0,2.5,10.2,9.1,3.3,...,7.2,6.9,14.4,0.0,5.6,NaN,NaN,NaN,NaN,NaN
54020,47.78,2021-02-28,20,7,6.0,0.0,2.5,10.2,9.1,3.3,...,7.2,6.9,14.4,0.0,5.6,NaN,NaN,NaN,NaN,NaN
54021,46.27,2021-02-28,21,7,6.0,0.0,2.5,10.2,9.1,3.3,...,7.2,6.9,14.4,0.0,5.6,NaN,NaN,NaN,NaN,NaN
54022,37.03,2021-02-28,22,7,6.0,0.0,2.5,10.2,9.1,3.3,...,7.2,6.9,14.4,0.0,5.6,NaN,NaN,NaN,NaN,NaN


In [71]:
#Se guarda el dataframe

# fh = open("df.pkl", "wb")
# pickle.dump(df, fh)
# fh.close()

In [6]:
#Se carga el dataframe

fh = open("df.pkl", "rb")
df = pickle.load(fh)
fh.close()

### Agrupamiento por provincias

In [7]:
#Se crea un diccionario nuevo que tiene como llaves cadenas de texto formadas por cada una de las provincias y las 4 variables que se analizan
#Los valores son todas las medicciones de esa variable de las estaciones situadas en esa provincia

dicc = {}

for i in set(estaciones_provincia.values()):      #Se recorren las provincias (valores únicos)
    for j in ["tmed", "prec", "velmedia", "sol"]: # Se recorren las variables meteorológicas
        llave = i + "-" + j                       #Cadena de texto (PROVINCIA-VARIABLE_METEOROLOGICA) que es la nueva llave
        dicc[llave] = np.array([])                #En la nueva llave se introduce como valor un vector vacío 
        for z in df.columns[4:]:                  #Se recorren los nombres de las columnas del dataframe
            estacion = z.split("-")[0]            #Del nombre de la columna, se escoge la primera parte que indica el indicativo de la estación
            variable = z.split("-")[1]            #Del nombre de la columna, se escoge la segunda parte que indica la variable meteorológica
            if (estaciones_provincia[estacion] == i) & (variable == j):
                dicc[llave] = np.append(dicc[llave], df.loc[:,z]) #Se va rellenando el vector con los valores de la columna del dataframe seleccionada

dicc

{'CASTELLON-tmed': array([8. , 8. , 8. , ..., 4.1, 4.1, 4.1]),
 'CASTELLON-prec': array([0. , 0. , 0. , ..., 0.9, 0.9, 0.9]),
 'CASTELLON-velmedia': array([1.7, 1.7, 1.7, ..., nan, nan, nan]),
 'CASTELLON-sol': array([nan, nan, nan, ..., nan, nan, nan]),
 'LLEIDA-tmed': array([nan, nan, nan, ..., 11., 11., 11.]),
 'LLEIDA-prec': array([0., 0., 0., ..., 0., 0., 0.]),
 'LLEIDA-velmedia': array([nan, nan, nan, ..., 1.7, 1.7, 1.7]),
 'LLEIDA-sol': array([nan, nan, nan, ..., 2.2, 2.2, 2.2]),
 'AVILA-tmed': array([ 9.,  9.,  9., ..., nan, nan, nan]),
 'AVILA-prec': array([ 0.,  0.,  0., ..., nan, nan, nan]),
 'AVILA-velmedia': array([0.8, 0.8, 0.8, ..., nan, nan, nan]),
 'AVILA-sol': array([nan, nan, nan, ..., nan, nan, nan]),
 'CEUTA-tmed': array([13.4, 13.4, 13.4, ..., 14. , 14. , 14. ]),
 'CEUTA-prec': array([ 0. ,  0. ,  0. , ..., 22.8, 22.8, 22.8]),
 'CEUTA-velmedia': array([3.9, 3.9, 3.9, ..., 6.7, 6.7, 6.7]),
 'CEUTA-sol': array([6.1, 6.1, 6.1, ..., 0. , 0. , 0. ]),
 'CUENCA-tmed': ar

In [8]:
#Comprobación de que cada vector del diccionario es divisible por el número de registros (54024) que hay en el dataframe original
#Así se comprueba que la anterior celda se ha ejecutado de manera correcta

for i in dicc:
    if len(dicc[i])%len(df) !=0:
        print("error")
    else:
        pass

In [25]:
#Se crea un nuevo dataframe que ya no contendrá todas las estaciones meteorológicas
#Sino la agrupación de ellas por provincias

df_provincias = df.iloc[:,0:4]
df_provincias

,value,date,time,weekday
0,65.41,2015-01-01,00,4
1,64.92,2015-01-01,01,4
2,64.48,2015-01-01,02,4
3,59.32,2015-01-01,03,4
4,56.04,2015-01-01,04,4
...,...,...,...,...
54019,42.19,2021-02-28,19,7
54020,47.78,2021-02-28,20,7
54021,46.27,2021-02-28,21,7
54022,37.03,2021-02-28,22,7


In [26]:
%%time

#Se crean las nuevas columnas del dataframe y se rellenan con los valores que proceden

for i in dicc:
    lista = []
    vector = dicc[i] #La variable vector contiene todos los valores de una variable meteorológica para una provincia
    series = int(len(dicc[i])/len(df)) #Se calcula cuántas veces hay 54024 registros (esto indica el número de estaciones que hay en una provincia)
    for j in range(len(df)):
        subvector = np.array([])
        for z in range(series):
            subvector = np.append(subvector, vector[j+len(df)*z]) #En el subvector se añaden los valores de cada estación meteorológica de una provincia en un determinado día y hora
        media = round(np.nanmean(subvector),2) #Se calcula la media
        lista.append(media) #Se añade la media a la lista
    df_provincias[i] = lista #Una vez que la lista tiene los 54024 registros para una variable y provincia, se añade al dataframe

df_provincias

<timed exec>:11: RuntimeWarning: Mean of empty slice


Wall time: 25min 37s


,value,date,time,weekday,A CORUÑA-tmed,A CORUÑA-prec,A CORUÑA-velmedia,A CORUÑA-sol,MURCIA-tmed,MURCIA-prec,...,TERUEL-velmedia,TERUEL-sol,CANTABRIA-tmed,CANTABRIA-prec,CANTABRIA-velmedia,CANTABRIA-sol,GIPUZKOA-tmed,GIPUZKOA-prec,GIPUZKOA-velmedia,GIPUZKOA-sol
0,65.41,2015-01-01,00,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
1,64.92,2015-01-01,01,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
2,64.48,2015-01-01,02,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
3,59.32,2015-01-01,03,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
4,56.04,2015-01-01,04,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54019,42.19,2021-02-28,19,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77
54020,47.78,2021-02-28,20,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77
54021,46.27,2021-02-28,21,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77
54022,37.03,2021-02-28,22,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77


### Se imputan los NaN

In [27]:
#Se comprueba cuantos NaN hay

valores_nulos = {}
for i in df_provincias.columns[4:]:
    if len(df_provincias[i].isnull().value_counts().values) == 2:  #Se comprueba si hay algún valor nulo
        valores_nulos[i] = df_provincias[i].isnull().value_counts().values[1]
        
valores_nulos

{'LLEIDA-prec': 24,
 'MELILLA-tmed': 120,
 'MELILLA-prec': 2232,
 'MELILLA-velmedia': 24,
 'MELILLA-sol': 96,
 'ARABA/ALAVA-prec': 1584,
 'ARABA/ALAVA-velmedia': 24,
 'HUELVA-sol': 216,
 'TOLEDO-sol': 216,
 'SEGOVIA-prec': 1224,
 'LUGO-prec': 6362,
 'LUGO-velmedia': 168,
 'LUGO-sol': 144,
 'GUADALAJARA-prec': 72,
 'GIRONA-prec': 72,
 'GIRONA-sol': 24,
 'AVILA-prec': 480,
 'AVILA-sol': 48,
 'CEUTA-tmed': 24,
 'CEUTA-velmedia': 144,
 'CEUTA-sol': 48,
 'HUESCA-sol': 840,
 'NAVARRA-sol': 3863,
 'PONTEVEDRA-prec': 504,
 'PONTEVEDRA-velmedia': 24,
 'ZAMORA-prec': 24,
 'JAEN-sol': 120,
 'VALLADOLID-prec': 503,
 'ALMERIA-sol': 24,
 'CASTELLON-prec': 24,
 'LA RIOJA-tmed': 48,
 'LA RIOJA-prec': 3096,
 'LA RIOJA-velmedia': 576,
 'LA RIOJA-sol': 96,
 'SORIA-prec': 5590,
 'SORIA-velmedia': 72,
 'PALENCIA-prec': 552,
 'PALENCIA-sol': 24,
 'CADIZ-sol': 24,
 'TARRAGONA-prec': 24,
 'LEON-prec': 24}

In [28]:
%%time

#Se sustituyen los NaN por el valor medio de la variable meteorológica en la misma estación en las 48 y 24 horas previas y posteriores

for i in range(df_provincias.shape[0]):
    for j in range(4, df_provincias.shape[1]):
        if np.isnan(df_provincias.iloc[i,j]):
            if i<24:
                valores = np.array([df_provincias.iloc[i+24,j], df_provincias.iloc[i+48,j]])
            elif 24<=i<48:
                valores = np.array([df_provincias.iloc[i-24,j], df_provincias.iloc[i+24,j], df_provincias.iloc[i+48,j]])
            elif 53976<=i<54000:
                valores = np.array([df_provincias.iloc[i-48,j], df_provincias.iloc[i-24,j], df_provincias.iloc[i+24,j]])
            elif i>=54000:
                valores = np.array([df_provincias.iloc[i-48,j], df_provincias.iloc[i-24,j]])
            else:
                valores = np.array([df_provincias.iloc[i-48,j], df_provincias.iloc[i-24,j], df_provincias.iloc[i+24,j], df_provincias.iloc[i+48,j]])
            media = round(np.nanmean(valores),2)
            df_provincias.iloc[i,j] = media
            
df_provincias

<timed exec>:16: RuntimeWarning: Mean of empty slice


Wall time: 7min 10s


,value,date,time,weekday,A CORUÑA-tmed,A CORUÑA-prec,A CORUÑA-velmedia,A CORUÑA-sol,MURCIA-tmed,MURCIA-prec,...,TERUEL-velmedia,TERUEL-sol,CANTABRIA-tmed,CANTABRIA-prec,CANTABRIA-velmedia,CANTABRIA-sol,GIPUZKOA-tmed,GIPUZKOA-prec,GIPUZKOA-velmedia,GIPUZKOA-sol
0,65.41,2015-01-01,00,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
1,64.92,2015-01-01,01,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
2,64.48,2015-01-01,02,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
3,59.32,2015-01-01,03,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
4,56.04,2015-01-01,04,4,9.21,0.0,1.52,7.56,8.75,0.00,...,1.67,8.10,7.36,0.00,1.87,7.93,4.82,0.00,1.25,8.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54019,42.19,2021-02-28,19,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77
54020,47.78,2021-02-28,20,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77
54021,46.27,2021-02-28,21,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77
54022,37.03,2021-02-28,22,7,10.96,0.0,4.24,8.90,12.59,0.16,...,2.90,4.07,7.07,1.57,3.33,7.23,8.68,1.27,2.05,2.77


In [26]:
#Se comprueba nuevamente cuantos NaN quedan

valores_nulos = {}
for i in df_provincias.columns[4:]:
    if len(df_provincias[i].isnull().value_counts().values) == 2:  #Se comprueba si hay algún valor nulo
        valores_nulos[i] = df_provincias[i].isnull().value_counts().values[1]
        
valores_nulos

{'HUELVA-sol': 168}

In [16]:
#Para los valores que aún quedan nulos, se hace imputación a partir del KNN
#No hay que incluir la variable "date", ya que al ser una cadena de texto daría error
#La imputación solo sustituye los NaN, por lo que no hay problema de "pisar" los datos ya existentes

imputer = KNNImputer(n_neighbors=250)
imputacion = pd.DataFrame(imputer.fit_transform(df_provincias.drop("date", axis = 1)))
imputacion.columns = df_provincias.drop("date", axis = 1).columns

for i in valores_nulos:
    df_provincias[i] = imputacion[i]

In [18]:
#Se observa que ya no quedan valores nulos en todo el dataframe

valores_nulos = {}
for i in df_provincias.columns[4:]:
    if len(df_provincias[i].isnull().value_counts().values) == 2:  #Se comprueba si hay algún valor nulo
        valores_nulos[i] = df_provincias[i].isnull().value_counts().values[1]
        
valores_nulos

{}

In [20]:
#Se guarda el dataframe df_provincias

# fh = open("df_provincias.pkl", "wb")
# pickle.dump(df_provincias, fh)
# fh.close()

In [13]:
#Se carga el dataframe df_provincias

fh = open("df_provincias.pkl", "rb")
df_provincias = pickle.load(fh)
fh.close()

### Creación dummies día de la semana

In [21]:
dummy_weekday = pd.get_dummies(df_provincias["weekday"])
dummy_weekday.columns = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]
dummy_weekday

,lunes,martes,miércoles,jueves,viernes,sábado,domingo
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
54019,0,0,0,0,0,0,1
54020,0,0,0,0,0,0,1
54021,0,0,0,0,0,0,1
54022,0,0,0,0,0,0,1


### Obtención datos de consumo

In [22]:
df_consumo = pd.read_csv("DemandaReal_2015-2021.csv", delimiter = ";")
df_consumo.drop(["id", "name", "geoid", "geoname", "datetime"], axis = 1, inplace = True)
df_consumo.columns = ["consumo"]
df_consumo = df_consumo.apply(lambda x : round(x, 2))
df_consumo

,consumo
0,25435.00
1,24511.50
2,22866.17
3,21392.83
4,20319.67
...,...
54019,27516.33
54020,29434.83
54021,29709.17
54022,28416.50


### Se añaden los dummies del día de la semana y los datos de consumo al dataframe de provincias

In [23]:
df_provincias = pd.concat([df_provincias, dummy_weekday, df_consumo], axis = 1)
df_provincias["time"] = df_provincias["time"].astype("int8")
df_provincias.drop("weekday", axis = 1, inplace = True)
df_provincias

,value,date,time,A CORUÑA-tmed,A CORUÑA-prec,A CORUÑA-velmedia,A CORUÑA-sol,MURCIA-tmed,MURCIA-prec,MURCIA-velmedia,...,GIPUZKOA-velmedia,GIPUZKOA-sol,lunes,martes,miércoles,jueves,viernes,sábado,domingo,consumo
0,65.41,2015-01-01,0,9.21,0.0,1.52,7.56,8.75,0.00,1.52,...,1.25,8.03,0,0,0,1,0,0,0,25435.00
1,64.92,2015-01-01,1,9.21,0.0,1.52,7.56,8.75,0.00,1.52,...,1.25,8.03,0,0,0,1,0,0,0,24511.50
2,64.48,2015-01-01,2,9.21,0.0,1.52,7.56,8.75,0.00,1.52,...,1.25,8.03,0,0,0,1,0,0,0,22866.17
3,59.32,2015-01-01,3,9.21,0.0,1.52,7.56,8.75,0.00,1.52,...,1.25,8.03,0,0,0,1,0,0,0,21392.83
4,56.04,2015-01-01,4,9.21,0.0,1.52,7.56,8.75,0.00,1.52,...,1.25,8.03,0,0,0,1,0,0,0,20319.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54019,42.19,2021-02-28,19,10.96,0.0,4.24,8.90,12.59,0.16,3.73,...,2.05,2.77,0,0,0,0,0,0,1,27516.33
54020,47.78,2021-02-28,20,10.96,0.0,4.24,8.90,12.59,0.16,3.73,...,2.05,2.77,0,0,0,0,0,0,1,29434.83
54021,46.27,2021-02-28,21,10.96,0.0,4.24,8.90,12.59,0.16,3.73,...,2.05,2.77,0,0,0,0,0,0,1,29709.17
54022,37.03,2021-02-28,22,10.96,0.0,4.24,8.90,12.59,0.16,3.73,...,2.05,2.77,0,0,0,0,0,0,1,28416.50


# Análisis exploratorio

In [24]:
#Tabla de correlaciones
#Al haber tantas variables no se aprecia a simple vista ninguna conclusión

correlacion = df_provincias.drop("date", axis = 1).corr()
correlacion

,value,time,A CORUÑA-tmed,A CORUÑA-prec,A CORUÑA-velmedia,A CORUÑA-sol,MURCIA-tmed,MURCIA-prec,MURCIA-velmedia,MURCIA-sol,...,GIPUZKOA-velmedia,GIPUZKOA-sol,lunes,martes,miércoles,jueves,viernes,sábado,domingo,consumo
value,1.000000,2.517155e-01,0.011518,-0.130192,-0.219215,0.020674,-0.032855,0.003908,-0.230602,-0.043690,...,-0.245939,0.033339,3.617771e-02,5.933809e-02,5.463600e-02,6.034571e-02,4.143776e-02,-8.191384e-02,-1.698269e-01,0.532123
time,0.251716,1.000000e+00,-0.000130,0.000024,0.000112,0.000069,-0.000052,0.000050,0.000132,0.000062,...,0.000099,0.000100,-7.233760e-19,-7.233760e-19,-7.233760e-19,-9.908947e-18,-1.023455e-17,-9.908947e-18,2.713386e-20,0.534860
A CORUÑA-tmed,0.011518,-1.295227e-04,1.000000,-0.230138,-0.173693,0.439008,0.864920,-0.050115,0.000920,0.499551,...,-0.236174,0.446675,3.007831e-03,2.213829e-03,-3.123458e-03,9.028369e-04,-7.412669e-03,-5.489938e-03,9.904286e-03,-0.072156
A CORUÑA-prec,-0.130192,2.359211e-05,-0.230138,1.000000,0.467658,-0.482281,-0.188264,-0.053446,-0.011733,-0.151708,...,0.302297,-0.166846,-4.997584e-02,-1.084813e-02,2.018131e-02,4.090693e-02,-1.572260e-02,1.474840e-02,6.572942e-04,0.014971
A CORUÑA-velmedia,-0.219215,1.123338e-04,-0.173693,0.467658,1.000000,-0.267162,-0.133516,0.006328,0.131775,-0.100723,...,0.410669,-0.162546,-1.275752e-02,6.504373e-03,6.838189e-03,6.232476e-03,-1.868618e-02,1.118230e-03,1.075118e-02,0.018629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jueves,0.060346,-9.908947e-18,0.000903,0.040907,0.006232,-0.022882,0.001611,0.014090,-0.007280,-0.007047,...,-0.004758,0.006213,-1.666233e-01,-1.666233e-01,-1.666233e-01,1.000000e+00,-1.669259e-01,-1.669259e-01,-1.669259e-01,0.115689
viernes,0.041438,-1.023455e-17,-0.007413,-0.015723,-0.018686,0.000927,0.002592,-0.009300,0.014165,-0.018962,...,-0.012452,-0.031456,-1.666233e-01,-1.666233e-01,-1.666233e-01,-1.669259e-01,1.000000e+00,-1.669259e-01,-1.669259e-01,0.085261
sábado,-0.081914,-9.908947e-18,-0.005490,0.014748,0.001118,0.012829,-0.003583,-0.000064,-0.005076,-0.003825,...,-0.016475,-0.004567,-1.666233e-01,-1.666233e-01,-1.666233e-01,-1.669259e-01,-1.669259e-01,1.000000e+00,-1.669259e-01,-0.161199
domingo,-0.169827,2.713386e-20,0.009904,0.000657,0.010751,0.014858,-0.002553,0.011056,0.017445,0.015723,...,0.028864,0.003017,-1.666233e-01,-1.666233e-01,-1.666233e-01,-1.669259e-01,-1.669259e-01,-1.669259e-01,1.000000e+00,-0.327726


In [26]:
#Análisis del sumatorio, mínimo y máximo de las correllaciones por variable meteorológica
#Se observa que el viento y, en menor medida, las precipitaciones son las variables meteorológicas con mayor correlación

tmed, tmed_min, tmed_max = 0, 1, 0
prec, prec_min, prec_max = 0, 1, 0
velmedia, velmedia_min, velmedia_max = 0, 1, 0
sol, sol_min, sol_max = 0, 1, 0

for i in correlacion.columns[2:-8]: #Se analizan únicamente las columnas que corresponden a variable meteorológicas
    if i.split("-")[1] == "tmed":
        valor = correlacion.loc["value", i]
        tmed = tmed + math.fabs(valor)
        if math.fabs(valor) < tmed_min:
            tmed_min = math.fabs(valor)
        if math.fabs(valor) > tmed_max:
            tmed_max = math.fabs(valor)
    if i.split("-")[1] == "prec":
        valor = correlacion.loc["value", i]
        prec = prec + math.fabs(valor)
        if math.fabs(valor) < prec_min:
            prec_min = math.fabs(valor)
        if math.fabs(valor) > prec_max:
            prec_max = math.fabs(valor)
    if i.split("-")[1] == "velmedia":
        valor = correlacion.loc["value", i]
        velmedia = velmedia + math.fabs(valor)
        if math.fabs(valor) < velmedia_min:
            velmedia_min = math.fabs(valor)
        if math.fabs(valor) > velmedia_max:
            velmedia_max = math.fabs(valor)
    if i.split("-")[1] == "sol":
        valor = correlacion.loc["value", i]
        sol = sol + math.fabs(valor)
        if math.fabs(valor) < sol_min:
            sol_min = math.fabs(valor)
        if math.fabs(valor) > sol_max:
            sol_max = math.fabs(valor)
        
print("Sumatorio correlación tmed (en valor absoluto):", tmed)
print("Correlación máxima tmed (en valor absoluto):", tmed_max)
print("Correlación mínima tmed (en valor absoluto):", tmed_min)
print("*******")
print("Sumatorio correlación prec (en valor absoluto):", prec)
print("Correlación máxima prec (en valor absoluto):", prec_max)
print("Correlación mínima prec (en valor absoluto):", prec_min)
print("*******")
print("Sumatorio correlación velmedia (en valor absoluto):", velmedia)
print("Correlación máxima velmedia (en valor absoluto):", velmedia_max)
print("Correlación mínima velmedia (en valor absoluto):", velmedia_min)
print("*******")
print("Sumatorio correlación sol (en valor absoluto):", sol)
print("Correlación máxima sol (en valor absoluto):", sol_max)
print("Correlación mínima sol (en valor absoluto):", sol_min)

Sumatorio correlación tmed (en valor absoluto): 1.0315711721854934
Correlación máxima tmed (en valor absoluto): 0.05168933159667274
Correlación mínima tmed (en valor absoluto): 0.0013622334999900795
*******
Sumatorio correlación prec (en valor absoluto): 3.7504690083261187
Correlación máxima prec (en valor absoluto): 0.16411592927340443
Correlación mínima prec (en valor absoluto): 0.002400308341555227
*******
Sumatorio correlación velmedia (en valor absoluto): 12.765936300324093
Correlación máxima velmedia (en valor absoluto): 0.38218409492043476
Correlación mínima velmedia (en valor absoluto): 0.045401643848181156
*******
Sumatorio correlación sol (en valor absoluto): 1.792642067792179
Correlación máxima sol (en valor absoluto): 0.12412981375453319
Correlación mínima sol (en valor absoluto): 0.0008739517708392682


In [27]:
#Se observan aquellas variables cuya correlación es mayor con la variable objetivo "value"
#Nuevamente se comprueba que el viento es la variable que más correlación

correlacion[(correlacion["value"] >=0.3) | (correlacion["value"] <= -0.3)][["value"]]

,value
value,1.000000
TOLEDO-velmedia,-0.311705
GRANADA-velmedia,-0.323919
ALBACETE-velmedia,-0.314047
MADRID-velmedia,-0.324654
VALENCIA-velmedia,-0.303086
AVILA-velmedia,-0.382184
ZAMORA-velmedia,-0.334181
VALLADOLID-velmedia,-0.319902
SALAMANCA-velmedia,-0.300222


In [28]:
#Dataframe con los datos más relevantes de las horas en las que se ha alcanzado valores máximos en el precio de la electricidad
#Se ve que estos precios suelen tener lugar por la tarde-noche (de 17 a 22) y en días muy fríos (la temperatura media del país raramente supera los 5ºC)
#También se observa que en estos momentos la velocidad del viento es relativamente baja (sobre 3 m/s) y que el consumo es alto

maximos = df_provincias.sort_values("value", ascending = False).head(50).index
df_maximos = pd.DataFrame(columns = ["Precio", "Día", "Hora", "Temperatura", "Precipitaciones", "Velocidad viento", "Horas sol", "Consumo"])
contador = 0

for i in maximos:
    precio = df_provincias.loc[i, "value"]
    dia = df_provincias.loc[i, "date"]
    hora = df_provincias.loc[i, "time"]
    tmed = np.array([])
    prec = np.array([])
    velmedia = np.array([])
    sol = np.array([])
    consumo = df_provincias.loc[i, "consumo"]
    for j in df_provincias.columns[3:-8]: #Se analizan únicamente las columnas que corresponden a variable meteorológicas
        if j.split("-")[1] == "tmed":
            tmed = np.append(tmed, df_provincias.loc[i,j])
        if j.split("-")[1] == "prec":
            prec = np.append(prec, df_provincias.loc[i,j])
        if j.split("-")[1] == "velmedia":
            velmedia = np.append(velmedia, df_provincias.loc[i,j])
        if j.split("-")[1] == "sol":
            sol = np.append(sol, df_provincias.loc[i,j])
    tmed = round(tmed.mean(),2)
    prec = round(prec.mean(),2)
    velmedia = round(velmedia.mean(),2)
    sol = round(sol.mean(),2)
    df_maximos.loc[contador] = [precio, dia, hora, tmed, prec, velmedia, sol, consumo]
    contador+=1
    
df_maximos

,Precio,Día,Hora,Temperatura,Precipitaciones,Velocidad viento,Horas sol,Consumo
0,133.28,2021-01-07,18,2.70,7.27,2.29,2.11,39546.67
1,130.11,2021-01-11,20,2.84,0.18,2.42,6.13,40883.33
2,129.58,2021-01-11,19,2.84,0.18,2.42,6.13,40468.83
3,128.90,2021-01-09,20,2.76,5.95,3.34,1.55,35874.17
4,128.87,2021-01-18,20,7.11,0.03,1.54,6.98,39302.83
5,127.91,2021-01-09,19,2.76,5.95,3.34,1.55,35121.17
6,127.86,2021-01-09,21,2.76,5.95,3.34,1.55,35718.67
7,127.83,2021-01-11,21,2.84,0.18,2.42,6.13,40236.83
8,126.53,2021-01-07,19,2.70,7.27,2.29,2.11,40009.00
9,125.57,2021-01-12,20,3.00,0.05,1.90,6.87,41564.17


In [29]:
df_maximos.mean()

Precio                120.0522
Hora                   17.7800
Temperatura             3.7624
Precipitaciones         4.9960
Velocidad viento        2.5620
Horas sol               3.7788
Consumo             38698.9330
dtype: float64

In [30]:
#Dataframe con los datos más relevantes de las horas en las que se ha alcanzado valores mínimos en el precio de la electricidad
#Los precios más bajos tienen lugar de madrugada, cuando el consumo es mínimo
#También se observa influencia de las temperaturas (suelen ser días moderadamente cálidos) y del viento (la velocidad suele situarse en torno a 5 m/s)

minimos = df_provincias.sort_values("value").head(50).index
df_minimos = pd.DataFrame(columns = ["Precio", "Día", "Hora", "Temperatura", "Precipitaciones", "Velocidad viento", "Horas sol", "Consumo"])
contador = 0

for i in minimos:
    precio = df_provincias.loc[i, "value"]
    dia = df_provincias.loc[i, "date"]
    hora = df_provincias.loc[i, "time"]
    tmed = np.array([])
    prec = np.array([])
    velmedia = np.array([])
    sol = np.array([])
    consumo = df_provincias.loc[i, "consumo"]
    for j in df_provincias.columns[3:-8]: #Se analizan únicamente las columnas que corresponden a variable meteorológicas
        if j.split("-")[1] == "tmed":
            tmed = np.append(tmed, df_provincias.loc[i,j])
        if j.split("-")[1] == "prec":
            prec = np.append(prec, df_provincias.loc[i,j])
        if j.split("-")[1] == "velmedia":
            velmedia = np.append(velmedia, df_provincias.loc[i,j])
        if j.split("-")[1] == "sol":
            sol = np.append(sol, df_provincias.loc[i,j])
    tmed = round(tmed.mean(),2)
    prec = round(prec.mean(),2)
    velmedia = round(velmedia.mean(),2)
    sol = round(sol.mean(),2)
    df_minimos.loc[contador] = [precio, dia, hora, tmed, prec, velmedia, sol, consumo]
    contador+=1
    
df_minimos

,Precio,Día,Hora,Temperatura,Precipitaciones,Velocidad viento,Horas sol,Consumo
0,4.51,2019-12-22,3,12.09,1.57,5.27,3.76,19814.50
1,4.61,2019-12-22,5,12.09,1.57,5.27,3.76,19353.00
2,4.77,2019-12-22,4,12.09,1.57,5.27,3.76,19402.17
3,4.81,2019-12-22,6,12.09,1.57,5.27,3.76,19739.67
4,5.01,2019-12-20,4,12.56,13.54,5.00,0.84,22637.83
5,5.18,2021-02-09,3,9.33,8.76,4.50,1.44,24049.00
6,5.26,2021-02-20,2,12.40,3.56,4.26,4.66,23492.33
7,5.31,2019-12-22,1,12.09,1.57,5.27,3.76,21762.33
8,5.33,2021-02-20,4,12.40,3.56,4.26,4.66,22579.33
9,5.36,2019-12-22,7,12.09,1.57,5.27,3.76,20630.83


In [31]:
df_minimos.mean()

Precio                  6.0270
Hora                    5.9000
Temperatura            11.2428
Precipitaciones         3.5684
Velocidad viento        4.7736
Horas sol               3.6424
Consumo             23325.8362
dtype: float64

In [32]:
#Dataframe con los datos más relevantes de los días en los que se ha alcanzado valores máximos en el precio de la electricidad
#Se observan conclusiones similares al análisis horario de los precios máximos

maximos = df_provincias.groupby("date").mean().reset_index().drop("time", axis = 1).sort_values("value", ascending = False).reset_index(drop = True)
df_maximos = pd.DataFrame(columns = ["Precio", "Día", "Temperatura", "Precipitaciones", "Velocidad viento", "Horas sol", "Consumo en el día"])

for i in range(25):
    precio = round(maximos.loc[i, "value"], 2)
    dia = maximos.loc[i, "date"]
    tmed = np.array([])
    prec = np.array([])
    velmedia = np.array([])
    sol = np.array([])
    consumo = round(maximos.loc[i, "consumo"]*24, 2) #Consumo durante todo el día
    for j in maximos.columns[2:-8]: #Se analizan únicamente las columnas que corresponden a variable meteorológicas
        if j.split("-")[1] == "tmed":
            tmed = np.append(tmed, maximos.loc[i,j])
        if j.split("-")[1] == "prec":
            prec = np.append(prec, maximos.loc[i,j])
        if j.split("-")[1] == "velmedia":
            velmedia = np.append(velmedia, maximos.loc[i,j])
        if j.split("-")[1] == "sol":
            sol = np.append(sol, maximos.loc[i,j])
    tmed = round(tmed.mean(),2)
    prec = round(prec.mean(),2)
    velmedia = round(velmedia.mean(),2)
    sol = round(sol.mean(),2)
    df_maximos.loc[i] = [precio, dia, tmed, prec, velmedia, sol, consumo]
    
df_maximos

,Precio,Día,Temperatura,Precipitaciones,Velocidad viento,Horas sol,Consumo en el día
0,103.35,2021-01-08,2.67,10.32,3.13,2.11,837438.82
1,102.93,2021-01-07,2.70,7.27,2.29,2.11,820038.17
2,99.20,2017-01-25,5.49,0.42,2.11,7.80,809042.16
3,97.02,2017-01-20,4.64,4.71,2.69,3.10,834010.17
4,95.90,2021-01-13,4.45,0.04,1.92,6.82,843962.00
5,95.41,2017-01-26,5.44,4.91,2.63,3.36,817144.34
6,95.32,2017-01-24,6.34,0.01,2.40,7.64,802147.52
7,93.77,2017-01-19,2.45,6.15,3.08,3.79,847353.82
8,93.12,2021-01-14,4.75,0.26,2.14,5.94,839743.83
9,91.51,2017-01-23,6.32,0.47,2.47,6.04,795999.17


In [33]:
df_maximos.mean()

Precio                   89.4992
Temperatura               5.8628
Precipitaciones           2.1924
Velocidad viento          2.4844
Horas sol                 5.4856
Consumo en el día    802335.9260
dtype: float64

In [34]:
#Dataframe con los datos más relevantes de los días en los que se ha alcanzado valores mínimos en el precio de la electricidad
#Se observan conclusiones similares al análisis horario de los precios mínimos

minimos = df_provincias.groupby("date").mean().reset_index().drop("time", axis = 1).sort_values("value").reset_index(drop = True)
df_minimos = pd.DataFrame(columns = ["Precio", "Día", "Temperatura", "Precipitaciones", "Velocidad viento", "Horas sol", "Consumo en el día"])

for i in range(25):
    precio = round(minimos.loc[i, "value"], 2)
    dia = minimos.loc[i, "date"]
    tmed = np.array([])
    prec = np.array([])
    velmedia = np.array([])
    sol = np.array([])
    consumo = round(minimos.loc[i, "consumo"]*24, 2) #Consumo durante todo el día
    for j in minimos.columns[2:-8]: #Se analizan únicamente las columnas que corresponden a variable meteorológicas
        if j.split("-")[1] == "tmed":
            tmed = np.append(tmed, minimos.loc[i,j])
        if j.split("-")[1] == "prec":
            prec = np.append(prec, minimos.loc[i,j])
        if j.split("-")[1] == "velmedia":
            velmedia = np.append(velmedia, minimos.loc[i,j])
        if j.split("-")[1] == "sol":
            sol = np.append(sol, minimos.loc[i,j])
    tmed = round(tmed.mean(),2)
    prec = round(prec.mean(),2)
    velmedia = round(velmedia.mean(),2)
    sol = round(sol.mean(),2)
    df_minimos.loc[i] = [precio, dia, tmed, prec, velmedia, sol, consumo]
    
df_minimos

,Precio,Día,Temperatura,Precipitaciones,Velocidad viento,Horas sol,Consumo en el día
0,6.97,2021-01-31,11.60,1.78,4.99,3.35,601236.64
1,7.27,2019-12-22,12.09,1.57,5.27,3.76,577408.33
2,8.90,2019-12-21,13.63,6.18,6.21,2.73,621883.32
3,8.95,2021-02-20,12.40,3.56,4.26,4.66,641205.66
4,9.50,2021-01-30,11.35,3.14,5.44,4.36,641001.32
5,11.47,2020-05-01,17.52,1.07,3.90,7.52,496358.16
6,12.72,2020-04-30,15.33,1.32,4.21,6.18,564335.49
7,13.51,2021-02-09,9.33,8.76,4.50,1.44,759459.17
8,14.48,2016-02-27,5.55,8.84,5.54,3.20,695601.32
9,14.51,2016-02-28,7.08,1.85,4.29,3.62,638542.83


In [35]:
df_minimos.mean()

Precio                   14.6080
Temperatura              12.0832
Precipitaciones           4.0276
Velocidad viento          4.3116
Horas sol                 4.7700
Consumo en el día    606258.6284
dtype: float64

### Análisis exploratorio con variación temporal

In [72]:
#Como el objetivo es predecir el precio del día de hoy no se puede partir de los datos de consumo de hoy, ya que no se tienen
#Tampoco se tienen los valores de las variables meteorológias para hoy
#Por ello se decide retrasar la varialbe "consumo" una semana, mientras que las variables meteorológicas se retrasan un día

df_temporal = df_provincias.iloc[168:,:3] #Los primeros 168 registros (correspondientes a los primeros 7 días) no se analizan

for i in df_provincias.columns[3:-8]:
    df_temporal[i] = list(df_provincias.loc[144:53999,i]) #Se obtienen las variable meteorológicas que van desde el día anterior al comienzo de la serie hasta el día anterior a la finalización de la serie

for i in df_provincias.columns[-8:-1]:
    df_temporal[i] = list(df_provincias.loc[168:,i]) #Se obtienen los dummies de los días de la semana

df_temporal["consumo"] = list(df_provincias.loc[:53855, "consumo"]) #Se obtienen los datos de consumo que, como se ha dicho anteriormente, van con una semana de retraso 
df_temporal.reset_index(drop = True, inplace = True)
df_temporal

,value,date,time,A CORUÑA-tmed,A CORUÑA-prec,A CORUÑA-velmedia,A CORUÑA-sol,MURCIA-tmed,MURCIA-prec,MURCIA-velmedia,...,GIPUZKOA-velmedia,GIPUZKOA-sol,lunes,martes,miércoles,jueves,viernes,sábado,domingo,consumo
0,73.73,2015-01-08,0,10.24,0.01,2.42,5.74,8.76,0.0,1.38,...,1.17,7.43,0,0,0,1,0,0,0,25435.00
1,70.99,2015-01-08,1,10.24,0.01,2.42,5.74,8.76,0.0,1.38,...,1.17,7.43,0,0,0,1,0,0,0,24511.50
2,68.30,2015-01-08,2,10.24,0.01,2.42,5.74,8.76,0.0,1.38,...,1.17,7.43,0,0,0,1,0,0,0,22866.17
3,64.22,2015-01-08,3,10.24,0.01,2.42,5.74,8.76,0.0,1.38,...,1.17,7.43,0,0,0,1,0,0,0,21392.83
4,63.53,2015-01-08,4,10.24,0.01,2.42,5.74,8.76,0.0,1.38,...,1.17,7.43,0,0,0,1,0,0,0,20319.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,42.19,2021-02-28,19,12.38,0.00,5.16,7.65,12.36,0.0,2.43,...,2.38,4.87,0,0,0,0,0,0,1,28735.50
53852,47.78,2021-02-28,20,12.38,0.00,5.16,7.65,12.36,0.0,2.43,...,2.38,4.87,0,0,0,0,0,0,1,30431.50
53853,46.27,2021-02-28,21,12.38,0.00,5.16,7.65,12.36,0.0,2.43,...,2.38,4.87,0,0,0,0,0,0,1,30433.67
53854,37.03,2021-02-28,22,12.38,0.00,5.16,7.65,12.36,0.0,2.43,...,2.38,4.87,0,0,0,0,0,0,1,29010.50


In [73]:
correlacion_temporal = df_temporal.drop("date", axis = 1).corr()
correlacion_temporal

,value,time,A CORUÑA-tmed,A CORUÑA-prec,A CORUÑA-velmedia,A CORUÑA-sol,MURCIA-tmed,MURCIA-prec,MURCIA-velmedia,MURCIA-sol,...,GIPUZKOA-velmedia,GIPUZKOA-sol,lunes,martes,miércoles,jueves,viernes,sábado,domingo,consumo
value,1.000000,2.514266e-01,0.016167,-0.166657,-0.209919,0.060470,-0.032479,0.016000,-0.146419,-0.034422,...,-0.182016,0.034314,3.578803e-02,5.950574e-02,5.396814e-02,6.107294e-02,4.150354e-02,-8.200865e-02,-1.696358e-01,0.482928
time,0.251427,1.000000e+00,-0.000180,-0.000079,0.000025,0.000092,-0.000103,-0.000002,0.000189,0.000052,...,0.000110,-0.000070,3.304531e-19,3.304531e-19,3.304531e-19,1.746745e-17,1.746745e-17,1.746745e-17,7.498685e-18,0.533232
A CORUÑA-tmed,0.016167,-1.796234e-04,1.000000,-0.232096,-0.177024,0.438905,0.864506,-0.050706,-0.001705,0.499763,...,-0.240033,0.447621,1.042159e-02,3.135751e-03,2.176316e-03,-4.372654e-03,1.216776e-03,-7.268716e-03,-5.288615e-03,-0.051027
A CORUÑA-prec,-0.166657,-7.885004e-05,-0.232096,1.000000,0.467252,-0.483648,-0.190729,-0.053728,-0.012959,-0.152481,...,0.301387,-0.167239,1.359113e-03,-5.005584e-02,-1.092019e-02,1.956058e-02,4.100378e-02,-1.577948e-02,1.475458e-02,0.012003
A CORUÑA-velmedia,-0.209919,2.523121e-05,-0.177024,0.467252,1.000000,-0.268884,-0.137667,0.005878,0.129405,-0.101430,...,0.409133,-0.162720,1.125613e-02,-1.350135e-02,6.206313e-03,5.752629e-03,7.254044e-03,-1.789288e-02,9.302527e-04,0.040347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jueves,0.061073,1.746745e-17,-0.004373,0.019561,0.005753,-0.016184,-0.000350,-0.023977,0.005052,0.014506,...,0.008865,0.034116,-1.666231e-01,-1.666231e-01,-1.666231e-01,1.000000e+00,-1.669267e-01,-1.669267e-01,-1.669267e-01,0.115717
viernes,0.041504,1.746745e-17,0.001217,0.041004,0.007254,-0.024059,0.001982,0.014118,-0.006680,-0.007024,...,-0.004593,0.005110,-1.666231e-01,-1.666231e-01,-1.666231e-01,-1.669267e-01,1.000000e+00,-1.669267e-01,-1.669267e-01,0.085230
sábado,-0.082009,1.746745e-17,-0.007269,-0.015779,-0.017893,0.001035,0.002939,-0.009309,0.014858,-0.019439,...,-0.012184,-0.032143,-1.666231e-01,-1.666231e-01,-1.666231e-01,-1.669267e-01,-1.669267e-01,1.000000e+00,-1.669267e-01,-0.161121
domingo,-0.169636,7.498685e-18,-0.005289,0.014755,0.000930,0.014072,-0.003430,-0.000089,-0.006004,-0.004558,...,-0.017313,-0.005749,-1.666231e-01,-1.666231e-01,-1.666231e-01,-1.669267e-01,-1.669267e-01,-1.669267e-01,1.000000e+00,-0.327582


In [74]:
correlacion_temporal[(correlacion_temporal["value"] >=0.25) | (correlacion_temporal["value"] <= -0.25)][["value"]]

,value
value,1.000000
time,0.251427
TOLEDO-velmedia,-0.252898
GRANADA-velmedia,-0.275132
MADRID-velmedia,-0.255996
LUGO-velmedia,-0.283802
AVILA-velmedia,-0.308810
ZAMORA-velmedia,-0.292446
VALLADOLID-velmedia,-0.266204
SALAMANCA-velmedia,-0.252506


# REGRESIÓN

## MinMaxScaler

In [75]:
#Se normalizan todas las variables entre 0 y 1

scaler = MinMaxScaler()
norm = scaler.fit_transform(df_temporal.drop(["date"], axis = 1)) #Se elimina "date" porque es una cadena de texto
df_escalado = pd.DataFrame(norm, columns = df_temporal.drop(["date"], axis = 1).columns)
df_escalado

,value,time,A CORUÑA-tmed,A CORUÑA-prec,A CORUÑA-velmedia,A CORUÑA-sol,MURCIA-tmed,MURCIA-prec,MURCIA-velmedia,MURCIA-sol,...,GIPUZKOA-velmedia,GIPUZKOA-sol,lunes,martes,miércoles,jueves,viernes,sábado,domingo,consumo
0,0.537548,0.000000,0.285848,0.000143,0.106087,0.399166,0.232100,0.0,0.089812,0.435507,...,0.066310,0.517049,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.351672
1,0.516269,0.043478,0.285848,0.000143,0.106087,0.399166,0.232100,0.0,0.089812,0.435507,...,0.066310,0.517049,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.315151
2,0.495379,0.086957,0.285848,0.000143,0.106087,0.399166,0.232100,0.0,0.089812,0.435507,...,0.066310,0.517049,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.250083
3,0.463695,0.130435,0.285848,0.000143,0.106087,0.399166,0.232100,0.0,0.089812,0.435507,...,0.066310,0.517049,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.191816
4,0.458337,0.173913,0.285848,0.000143,0.106087,0.399166,0.232100,0.0,0.089812,0.435507,...,0.066310,0.517049,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.149376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,0.292615,0.826087,0.386129,0.000000,0.344348,0.531989,0.354259,0.0,0.230563,0.198551,...,0.195722,0.338900,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.482197
53852,0.336025,0.869565,0.386129,0.000000,0.344348,0.531989,0.354259,0.0,0.230563,0.198551,...,0.195722,0.338900,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.549269
53853,0.324299,0.913043,0.386129,0.000000,0.344348,0.531989,0.354259,0.0,0.230563,0.198551,...,0.195722,0.338900,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.549355
53854,0.252543,0.956522,0.386129,0.000000,0.344348,0.531989,0.354259,0.0,0.230563,0.198551,...,0.195722,0.338900,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.493073


In [76]:
#Se separa la variable objeto (y) del resto (X)

X = df_escalado.iloc[:,1:]
y = df_escalado.iloc[:,0]

In [77]:
#Se crea un diccionario donde se almacenaran los r2 ajustados obtenidos

resultados_r2 = {}

## Árbol de Decisión

In [93]:
%%time

#Se prueba el regresor del árbol de clasificación (por defecto)
#Como método de validación se hace Hold-out (100 veces)

r2_ajustada_arbol_defecto = []
error_arbol_defecto = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    y_test = np.array(y_test)
    
    reg = DecisionTreeRegressor()
    reg = reg.fit(X_train, y_train)
    yhat = reg.predict(X_test)
    
    suma = 0
    for i in range(len(yhat)):
        suma = suma + math.fabs(yhat[i] - y_test[i])
    error_arbol_defecto.append(suma/len(yhat))
        
    r2 = 1 - (1-r2_score(y_test,yhat))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    r2_ajustada_arbol_defecto.append(r2)

Wall time: 9min 57s


In [78]:
#Se hace GridSearch al árbol de decisión, para ver qué parámetros pueden hacer que se mejore el resultado anterior

params={'criterion':["mse", "friedman_mse", "mae", "poisson"],
        'max_depth': [20, 50, 100],# Maxima pofundidad del arbol
        'max_leaf_nodes': [50, 100, 200, 500], # maximo de nodos del arbol
        'min_samples_split': [2,5] # The minimum number of samples required to split an internal node
        }

reg = DecisionTreeRegressor()
grid_solver = GridSearchCV(estimator = reg,
                   param_grid = params,
                   #scoring=scorers,
                   cv = 5,
                   #refit="accuracy",
                   n_jobs=-1)

model_result = grid_solver.fit(X_train,y_train)
print(model_result.best_score_)
print(model_result.best_params_)

0.7587925676654613
{'criterion': 'friedman_mse', 'max_depth': 100, 'max_leaf_nodes': 500, 'min_samples_split': 5}


In [95]:
%%time

#Se prueba el regresor del árbol de clasificación con los parámetros obtenidos del GridSearch anterior
#Como método de validación se hace Hold-out (100 veces)

r2_ajustada_arbol_ajustado = []
error_arbol_ajustado = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    y_test = np.array(y_test)
    
    reg = DecisionTreeRegressor(criterion = "friedman_mse",
                                max_depth = 100,
                                max_leaf_nodes = 500,
                                min_samples_split = 5)
    reg = reg.fit(X_train, y_train)
    yhat = reg.predict(X_test)
    
    suma = 0
    for i in range(len(yhat)):
        suma = suma + math.fabs(yhat[i] - y_test[i])
    error_arbol_ajustado.append(suma/len(yhat))
        
    r2 = 1 - (1-r2_score(y_test,yhat))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    r2_ajustada_arbol_ajustado.append(r2)

## Random Forest

In [96]:
%%time

#Se realiza la regresión con Random Forest por defecto
#Como método de validación se hace Hold-out (100 veces)

r2_ajustada_forest_defecto = []
error_forest_defecto = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    y_test = np.array(y_test)

    reg = RandomForestRegressor()
    reg = reg.fit(X_train, y_train)
    yhat = reg.predict(X_test)

    suma = 0
    for i in range(len(yhat)):
        suma = suma + math.fabs(yhat[i] - y_test[i])
    error_forest_defecto.append(suma/len(yhat))
        
    r2 = 1 - (1-r2_score(y_test,yhat))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    r2_ajustada_forest_defecto.append(r2)

Wall time: 10h 30min 49s


## AdaBoost

In [97]:
%%time

#Se realiza la regresión con AdaBoost por defecto
#Como método de validación se hace Hold-out (100 veces)

r2_ajustada_adaboost_defecto = []
error_adaboost_defecto = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    y_test = np.array(y_test)
    
    reg = AdaBoostRegressor()
    reg = reg.fit(X_train, y_train)
    yhat = reg.predict(X_test)

    suma = 0
    for i in range(len(yhat)):
        suma = suma + math.fabs(yhat[i] - y_test[i])
    error_adaboost_defecto.append(suma/len(yhat))
        
    r2 = 1 - (1-r2_score(y_test,yhat))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    r2_ajustada_adaboost_defecto.append(r2)

Wall time: 1h 29min 1s


In [100]:
params={#'base_estimator':[None],
        'n_estimators': [25, 50, 100, 175, 250],
        'learning_rate': [0.2, 0.5, 0.8, 1, 2],
        'loss': ["linear", "square", "exponential"]
        #'random_state' : [None],
        }

reg = AdaBoostRegressor()
grid_solver = GridSearchCV(estimator = reg,
                   param_grid = params,
                   #scoring=scorers,
                   cv = 5,
                   #refit="accuracy",
                   n_jobs=-1)

model_result = grid_solver.fit(X_train,y_train)
print(model_result.best_score_)
print(model_result.best_params_)

0.435196072080472
{'learning_rate': 2, 'loss': 'exponential', 'n_estimators': 100}


In [113]:
%%time

#Se realiza la regresión con AdaBoost con los parámetros del GridSearch anterior
#Como método de validación se hace Hold-out (100 veces)

r2_ajustada_adaboost_ajustado = []
error_adaboost_ajustado = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    y_test = np.array(y_test)
    
    reg = AdaBoostRegressor(learning_rate = 2,
                            loss = 'exponential',
                            n_estimators = 100)
    reg = reg.fit(X_train, y_train)
    yhat = reg.predict(X_test)

    suma = 0
    for i in range(len(yhat)):
        suma = suma + math.fabs(yhat[i] - y_test[i])
    error_adaboost_ajustado.append(suma/len(yhat))
        
    r2 = 1 - (1-r2_score(y_test,yhat))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    r2_ajustada_adaboost_ajustado.append(r2)

Wall time: 2h 55min 39s


## GradientBoosting

In [100]:
%%time

#Se realiza la regresión con AdaBoost con los parámetros del GridSearch anterior
#Como método de validación se hace Hold-out (100 veces)

r2_ajustada_gradientboosting_defecto = []
error_gradientboosting_defecto = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    y_test = np.array(y_test)
    
    reg = GradientBoostingRegressor()
    reg = reg.fit(X_train, y_train)
    yhat = reg.predict(X_test)

    suma = 0
    for i in range(len(yhat)):
        suma = suma + math.fabs(yhat[i] - y_test[i])
    error_gradientboosting_defecto.append(suma/len(yhat))
        
    r2 = 1 - (1-r2_score(y_test,yhat))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    r2_ajustada_gradientboosting_defecto.append(r2)

Wall time: 2h 42min 36s


## Resultados

In [106]:
#Árbol de regresión por defecto

print("Media r2 árbol defecto:", np.array(r2_ajustada_arbol_defecto).mean())
print("Desviación típica r2 árbol defecto:", np.array(r2_ajustada_arbol_defecto).std())
print("Media error árbol defecto:", np.array(error_arbol_defecto).mean())
print("Desviación típica error árbol defecto:", np.array(error_arbol_defecto).std())

Media r2 árbol defecto: 0.901066488815569
Desviación típica r2 árbol defecto: 0.004364841486324276
Media error árbol defecto: 0.023573680349402628
Desviación típica error árbol defecto: 0.0003542729957171238


In [108]:
#Árbol de regresión ajustado (Grid Search)

print("Media r2 árbol ajustado:", np.array(r2_ajustada_arbol_ajustado).mean())
print("Desviación típica r2 árbol ajustado:", np.array(r2_ajustada_arbol_ajustado).std())
print("Media error árbol ajustado:", np.array(error_arbol_ajustado).mean())
print("Desviación típica error árbol ajustado:", np.array(error_arbol_ajustado).std())

Media r2 árbol ajustado: 0.7653165212574251
Desviación típica r2 árbol ajustado: 0.008374769604381495
Media error árbol ajustado: 0.04373382314039901
Desviación típica error árbol ajustado: 0.0006969953181149631


In [109]:
#Random Forest por defecto

print("Media r2 random forest defecto:", np.array(r2_ajustada_forest_defecto).mean())
print("Desviación típica r2 random forest defecto:", np.array(r2_ajustada_forest_defecto).std())
print("Media error random forest defecto:", np.array(error_forest_defecto).mean())
print("Desviación típica error random forest defecto:", np.array(error_forest_defecto).std())

Media r2 random forest defecto: 0.9510954262314918
Desviación típica r2 random forest defecto: 0.0013276667560953932
Media error random forest defecto: 0.017726552408787143
Desviación típica error random forest defecto: 0.00018363577256678146


In [110]:
#AdaBoost por defecto

print("Media r2 adaboost defecto:", np.array(r2_ajustada_adaboost_defecto).mean())
print("Desviación típica r2 adaboost defecto:", np.array(r2_ajustada_adaboost_defecto).std())
print("Media error adaboost defecto:", np.array(error_adaboost_defecto).mean())
print("Desviación típica error adaboost defecto:", np.array(error_adaboost_defecto).std())

Media r2 adaboost defecto: 0.4125922199560068
Desviación típica r2 adaboost defecto: 0.009935166466070452
Media error adaboost defecto: 0.07518939555555373
Desviación típica error adaboost defecto: 0.0006450472654686071


In [115]:
#AdaBoost ajustado (Grid Search)

print("Media r2 adaboost ajustado:", np.array(r2_ajustada_adaboost_ajustado).mean())
print("Desviación típica r2 adaboost ajustado:", np.array(r2_ajustada_adaboost_ajustado).std())
print("Media error adaboost ajustado:", np.array(error_adaboost_ajustado).mean())
print("Desviación típica error adaboost ajustado:", np.array(error_adaboost_ajustado).std())

Media r2 adaboost ajustado: 0.43300465258363713
Desviación típica r2 adaboost ajustado: 0.013497720785307624
Media error adaboost ajustado: 0.07449125930736387
Desviación típica error adaboost ajustado: 0.0007794783889869386


In [112]:
#GradientBoosting por defecto

print("Media r2 gradientboosting defecto:", np.array(r2_ajustada_gradientboosting_defecto).mean())
print("Desviación típica r2 gradientboosting defecto:", np.array(r2_ajustada_gradientboosting_defecto).std())
print("Media error gradientboosting defecto:", np.array(error_gradientboosting_defecto).mean())
print("Desviación típica error gradientboosting defecto:", np.array(error_gradientboosting_defecto).std())

Media r2 gradientboosting defecto: 0.665128458679505
Desviación típica r2 gradientboosting defecto: 0.004726586542932418
Media error gradientboosting defecto: 0.05389828725962392
Desviación típica error gradientboosting defecto: 0.00042016578352627176
